In [1]:
def composition(k, text):
    composition_list = []
    for i in range(len(text)-k+1):
        composition_list.append(text[i:i+k])
    #composition_list.sort()
    return composition_list

In [23]:
a = composition(100, dna_data[1])

In [292]:
adjency_list_degree_list(['1->2,3,5','2->4','3->2,5','4->1,2,5','5->3'])

defaultdict(list,
            {'1': [3, 1], '2': [1, 3], '3': [2, 2], '4': [3, 1], '5': [1, 3]})

In [2]:
def genome_path(patterns):
    genome_path = ''
    genome_path += patterns[0]
    for i in range(len(patterns)-1):
        genome_path += patterns[i+1][-1]
    return genome_path
        
    

In [4]:
genome_path(sample_patterns)

'AAATGCCCACACACGATATCGCTA'

In [6]:
from collections import defaultdict

def overlap_graph(patterns):
    overlapGraph = defaultdict(list)
    for i in range(len(patterns)):
        for j in range(len(patterns)):
            if i != j and patterns[i][1:] == patterns[j][:-1]:
                overlapGraph[i].append(j)
    return overlapGraph

In [17]:
og_dict = overlap_graph(dna_data)

In [ ]:
for x, y in og_dict.items():
    for y_ in y:
        print("{} -> {}".format(dna_data[x], dna_data[y_]))

In [7]:
def deBrujin(text, k):
    patterns = [text[i:i+k-1] for i in range(len(text)-k+2)]
    debrujin_dict = defaultdict(list)
    for i in range(len(patterns)-1):
        debrujin_dict[patterns[i]].append(patterns[i+1])
    data = []
    for x, y in debrujin_dict.items():
        data.append("{} -> {}".format(x, ", ".join(y)))
    return data


    

In [8]:
deBrujin('TAATGCCATGGGATGTT', 2)

['T -> A, G, G, G, T', 'A -> A, T, T, T', 'G -> C, G, G, A, T', 'C -> C, A']

In [11]:
deBrujin('TAATGCCATGGGATGTT', 3)

['TA -> AA',
 'AA -> AT',
 'AT -> TG, TG, TG',
 'TG -> GC, GG, GT',
 'GC -> CC',
 'CC -> CA',
 'CA -> AT',
 'GG -> GG, GA',
 'GA -> AT',
 'GT -> TT']

In [10]:
deBrujin('TAATGGGATGCCATGTT', 3)

['TA -> AA',
 'AA -> AT',
 'AT -> TG, TG, TG',
 'TG -> GG, GC, GT',
 'GG -> GG, GA',
 'GA -> AT',
 'GC -> CC',
 'CC -> CA',
 'CA -> AT',
 'GT -> TT']

In [12]:
def deBrujin_from_kmers(kmers):
    debrujin_dict = defaultdict(list)
    for x in kmers:
        debrujin_dict[x[:-1]].append(x[1:])
    data = []
    for x, y in debrujin_dict.items():
        data.append("{} -> {}".format(x, ", ".join(y)))
    return data

In [13]:
deBrujin_from_kmers(['CTTA','ACCA','TACC','GGCT','GCTT','TTAC'])

['CTT -> TTA',
 'ACC -> CCA',
 'TAC -> ACC',
 'GGC -> GCT',
 'GCT -> CTT',
 'TTA -> TAC']

In [14]:
def adjency_list_to_dict(adjency_list):
    adjency_dict = {}
    for x in adjency_list:
        x = x.replace(" ","")
        start, ends = x.split('->')[0], x.split('->')[1].split(',')
        adjency_dict[start] = ends
    return adjency_dict

from collections import Counter
from collections import defaultdict

# return list that contains indegree and outdegree of each vertex
def adjency_list_degree_list(adjency_list):
    adjency_dict = adjency_list_to_dict(adjency_list)
    degree_list = defaultdict(list)
    indegree_count = Counter([x for a in adjency_dict.values() for x in a])
    for x in adjency_dict.keys():
        degree_list[x].append(len(adjency_dict[x]))
        degree_list[x].append(indegree_count[x])
    
    end_points = set([x for a in adjency_dict.values() for x in a])
    start_points = set(adjency_dict.keys())
    no_outgoing_points = end_points.difference(start_points)
    
    for a in no_outgoing_points:
        degree_list[a].append(0)
        degree_list[a].append(indegree_count[a])
    
    return degree_list

# return left number of edges
def adjency_dict_value_length(adjency_dict):
    a = 0
    for x in adjency_dict.values():
        a += len(x)
    return a

# check if the graph has eulerian cycle (input : difference of degrees in each point) 
def has_eulerian_cycle(x):
    x.sort()
    if x == [0]*len(x):
        return True
    elif x == [-1] + [0]*(len(x)-2) + [1]:
        return True
    else:
        return False

In [15]:
sample = '''0->2
1->3
2->1
3->0,4
6->3,7
7->8
8->9
9->6'''

In [16]:
import random

def euler_cycle(data):
    adjency_dict = adjency_list_to_dict(data)
    degree_list = adjency_list_degree_list(data)
    stack = []
    circuit = []
    
    #check if graph has eulerian cycle, then set the start vertex
    degree_diff_list = [x[0]-x[1] for x in degree_list.values()] 
    if not has_eulerian_cycle(degree_diff_list):
        print("There's no eulerian cycle in this graph")
        return 
    elif 1 in degree_diff_list:
        for i, x in enumerate(degree_diff_list): 
            if x == 1:
                start_index = i
        start_vertex = list(adjency_dict.keys())[start_index]
    else:
        start_vertex = random.choice(list(adjency_dict.keys()))
    
    location = start_vertex
    while adjency_dict_value_length(adjency_dict) != 0:
        if adjency_dict[location]: # if there are outgoing neighbors
            next_location = random.choice(adjency_dict[location])  # go to that neighbor vertex
            stack.append(location)  # add last vertex to stack
            adjency_dict[location].remove(next_location)  # remove the edge
            location = next_location  # update location
        else:  # if there are no outgoing neighbors
            circuit.append(location)
            location = stack.pop()
    
    while stack:
        circuit.append(location)
        location = stack.pop()
    circuit.append(location)
    
    
    
    return circuit[::-1]

In [17]:
def euler_path(data):
    adjency_dict = adjency_list_to_dict(data)
    degree_list = adjency_list_degree_list(data)
    stack = []
    circuit = []
    
    # find start vertex and end vertex
#    end_points = set([x for a in adjency_dict.values() for x in a])
#    start_points = set(adjency_dict.keys())
    
    degree_diff_list = [x[0]-x[1] for x in degree_list.values()]
    
    for i, x in enumerate(degree_diff_list): 
        if x == 1:
            start_index = i
    start_vertex = list(adjency_dict.keys())[start_index]
    
    # decide end vertex
    end_vertex = None
    for node, degrees in degree_list.items():
        if degrees[0] == 0:
            end_vertex = node

    #if end_vertex_set:  # if there are only 0s and 1 (no -1) 
    #    (end_vertex, ) = end_vertex_set

    if end_vertex:
        # make dummy-edge end vertex and start vertex
        adjency_dict[end_vertex] = [start_vertex]

    else : # if there is both -1 and 1
        for i, x in enumerate(degree_diff_list): 
            if x == -1:
                end_index = i

        end_vertex = list(adjency_dict.keys())[end_index] 
        adjency_dict[end_vertex].append(start_vertex)
    
    location = start_vertex
    while adjency_dict_value_length(adjency_dict) != 0:
        if adjency_dict[location]: # if there are outgoing neighbors
            next_location = random.choice(adjency_dict[location])  # go to that neighbor vertex
            stack.append(location)  # add last vertex to stack
            adjency_dict[location].remove(next_location)  # remove the edge
            location = next_location  # update location
            print(stack)
        else:  # if there are no outgoing neighbors
            circuit.append(location)
            location = stack.pop()
            print(stack)
        print(adjency_dict_value_length(adjency_dict))
    
    while stack:
        circuit.append(location)
        location = stack.pop()
    circuit.append(location)
    circuit = circuit[::-1]
    
    # correct the circuit (there are multiple eulerian cycles that are not right) and remove dummy edge
    if circuit[-2:] == [end_vertex, start_vertex]:
        real_circuit = circuit
    else:
        for i in range(len(circuit)-1):
            if circuit[i:i+2] == [end_vertex, start_vertex]:
                cut_index = i
        real_circuit = circuit[cut_index+1:-1] + circuit[:cut_index+2] 
                
    
    
    return real_circuit[:-1]

        

In [20]:
def genome_reconstruction(data):
    # data : list of kmer fractions 
    return genome_path(euler_path(deBrujin_from_kmers(data)))

In [78]:
k_uni = [''.join(x) for x in itertools.product('01', repeat=3)]

In [79]:
k_uni_data = deBrujin_from_kmers(k_uni)
k_uni_deg_list = adjency_list_degree_list(k_uni_data)
k_uni_deg_list

defaultdict(list, {'00': [2, 2], '01': [2, 2], '10': [2, 2], '11': [2, 2]})

In [22]:
import itertools

def k_universal_strings(k):
    k_digit_binary_list = [''.join(x) for x in itertools.product('01', repeat=k)]
    k_universal_debrujin = deBrujin_from_kmers(k_digit_binary_list)
    a = euler_cycle(k_universal_debrujin)
    
    # cause it is circular string, and euler cycle we made includes extra nodes in the end,
    # we have to remove k-1 elements in the end
    for i in range(k-1):
        a.pop()
    return genome_path(a)

    

In [21]:
k_universal_strings(8)

'1101111011010001011010100011011000100100011110100101010011101000001110110011001110011011111111001111110101010111100001101011001001101001100011000001000100001001111001000000101011011011100000000110010110000101110010100001111100011100010100100101111101110101'

In [23]:
def k_d_mer_composition(text, k, d):
    compositions = []
    for i in range(len(text)-2*k-d+1):
        compositions.append("({}|{})".format(text[i:i+k],text[i+k+d:i+2*k+d]))
    #compositions.sort()
    return compositions

In [24]:
k_d_mer_composition('TAATGCCATGGGATGTT', 4, 1)

['(TAAT|CCAT)',
 '(AATG|CATG)',
 '(ATGC|ATGG)',
 '(TGCC|TGGG)',
 '(GCCA|GGGA)',
 '(CCAT|GGAT)',
 '(CATG|GATG)',
 '(ATGG|ATGT)',
 '(TGGG|TGTT)']

In [25]:
import re

# input : list of debrujin graph (eulerian path) 
def string_spelled_by_gapped_patterns(gapped_patterns, k, d):

    prefix_list = []
    suffix_list = []
    
    for x in gapped_patterns:
        splitted = re.split('[|]+', x)
        prefix_list.append(splitted[0])
        suffix_list.append(splitted[1])
    
    prefix_string = genome_path(prefix_list)
    
    #print(prefix_string)
    suffix_string = genome_path(suffix_list)
    #print(suffix_string)
    
    if prefix_string[k+d:] != suffix_string[:-(k+d)]:
        return "There is no string spelled by the gapped patterns"
    else:
        whole_string = prefix_string + suffix_string[-(k+d):]
        return whole_string
    
    

In [26]:
def prefix(gapped_pattern):
    splitted = re.split('[|]+', gapped_pattern)
    return splitted[0][:-1]+'|'+splitted[1][:-1]

def suffix(gapped_pattern):
    splitted = re.split('[|]+', gapped_pattern)
    return splitted[0][1:]+'|'+splitted[1][1:]

def make_debrujin_edge_from_read_pair(data):
    return "{} -> {}".format(prefix(data), suffix(data))

# first create euler path than create genome path (input is unordered gapped patterns)
def create_genome_path_from_gapped_patterns(gapped_patterns, k, d):
    
    all_edges = []
    
    for x in gapped_patterns:
        all_edges.append(make_debrujin_edge_from_read_pair(x))
    
    print(all_edges)
        
    return string_spelled_by_gapped_patterns(euler_path(all_edges), k, d)
    
   

    

In [27]:
s_data = ['ACC|ATA','ACT|ATT','ATA|TGA','ATT|TGA','CAC|GAT','CCG|TAC','CGA|ACT','CTG|AGC','CTG|TTC','GAA|CTT','GAT|CTG','GAT|CTG','TAC|GAT','TCT|AAG','TGA|GCT','TGA|TCT','TTC|GAA']

In [29]:
# data : path information
def maximal_non_branching_paths(data):
    adjency_dict = adjency_list_to_dict(data)
    degree_list = adjency_list_degree_list(data)
    paths = []
    for v, w_list in adjency_dict.items():
        if degree_list[v] != [1,1]: # if v is not a 1-in-1-out node 
            if degree_list[v][0] > 0:  # if out(v) > 0
                for w in w_list:
                    non_branching_path = [v, w]
                    if v == w:
                        path.append(non_branching_path)
                    while degree_list[w] == [1,1]:
                        non_branching_path.append(adjency_dict[w][0])
                        w = adjency_dict[w][0]
                    paths.append(non_branching_path)

    cycle = []
    
    for v, w_list in adjency_dict.items():
        if degree_list[v] == [1,1]:
            w = adjency_dict[v][0]
            cycle_candidates = [v, w]
            if degree_list[w] == [1,1]:
                still_in_cycle = True

                while still_in_cycle:
                    if degree_list[adjency_dict[w][0]][0] == 0:
                        still_in_cycle = False 
                    else:
                        cycle_candidates.append(adjency_dict[w][0])
                        w = adjency_dict[w][0]
                    if w == v:
                        cycle.append(cycle_candidates)
                        still_in_cycle = False
    
    set_list = []
    for a in cycle:
        if set(a) not in set_list:
            paths.append(a)
            set_list.append(set(a))
            
    path_strings = []
    for a in paths:
        path_strings.append(' -> '.join(a))
        
    return path_strings

In [33]:
# input : list of kmers -> output : possible contigs (maximal nonbranching paths in debrujin graph)
def contigs_from_kmers(kmer_patterns):
    data = deBrujin_from_kmers(kmer_patterns)
    contig_paths = maximal_non_branching_paths(data)
    contigs = []
    for a in contig_paths:
        nodes = a.split(' -> ')
        contigs.append(genome_path(nodes))
    return contigs